In [1]:
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install sentence-transformers

%pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-readers-web

%pip install llama-index-embeddings-openai
%pip install llama-index-embeddings-adapter
%pip install llama-index-finetuning


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing in

In [2]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM


import os
import warnings

warnings.filterwarnings('ignore')

In [3]:
training_file_name = "Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf"
validation_file_name = "Unraveling_the_Mystery_of_Scaling_Laws.pdf"

working_dir = "./"

!wget 'https://arxiv.org/pdf/2402.04177.pdf'  -O "Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf"
!wget 'https://arxiv.org/pdf/2403.06563.pdf' -O "Unraveling_the_Mystery_of_Scaling_Laws.pdf"

TRAIN_FILES = [os.path.join(working_dir, training_file_name)]
TRAIN_CORPUS_FPATH = "./train_corpus.json"

VAL_FILES = [os.path.join(working_dir, validation_file_name)]
VAL_CORPUS_FPATH = "./val_corpus.json"

--2025-03-28 13:13:14--  https://arxiv.org/pdf/2402.04177.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.3.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2402.04177 [following]
--2025-03-28 13:13:14--  http://arxiv.org/pdf/2402.04177
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1579680 (1.5M) [application/pdf]
Saving to: ‘Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf’

Scaling_Laws_for_Do 100%[===================>]   1.51M  --.-KB/s    in 0.01s   

2025-03-28 13:13:14 (105 MB/s) - ‘Scaling_Laws_for_Downstream_Task_Performance_of_Large_Language_Models.pdf’ saved [1579680/1579680]

--2025-03-28 13:13:14--  https://arxiv.org/pdf/2403.06563.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.131.42, 151.101.67.42, ...
Connect

In [23]:
print("🔹 First 5 Training Nodes:")
for i, node in enumerate(train_nodes[:5]):
    print(f"Node {i+1}: {node.text[:500]}...\n")  # Print first 500 chars

print("\n🔹 First 5 Validation Nodes:")
for i, node in enumerate(val_nodes[:5]):
    print(f"Node {i+1}: {node.text[:500]}...\n")


🔹 First 5 Training Nodes:
Node 1: Published as a conference paper at ICLR 2025
SCALING LAWS FOR DOWNSTREAM TASK
PERFORMANCE IN MACHINE TRANSLATION
Berivan Isik♣, Natalia Ponomareva♣, Hussein Hazimeh♢∗, Dimitris Paparas♣
Sergei Vassilvitskii♣, Sanmi Koyejo§∗
♣Google Research, ♢OpenAI, §Stanford University
berivan@google.com
ABSTRACT
Scaling laws provide important insights that can guide the design of large language
models (LLMs). Existing work has primarily focused on studying scaling laws for
pretraining (upstream) loss. However...

Node 2: Published as a conference paper at ICLR 2025
performance after finetuning on the task. We find that, in addition to the finetuning data size and the
choice of the performance metric, this relation fundamentally depends on the alignment between
the pretraining data and the downstream task (based on the translation alignment score provided in
Section 3). While similar observations have been made in different contexts in the transfer learning
literatur

In [4]:
import json
#generating synthetic data
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode

def load_corpus(files):
    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=True)
    print(f"Parsed {len(nodes)} nodes")
    return nodes

train_nodes = load_corpus(TRAIN_FILES)
val_nodes = load_corpus(VAL_FILES)

Parsing nodes:   0%|          | 0/22 [00:00<?, ?it/s]

Parsed 30 nodes


Parsing nodes:   0%|          | 0/13 [00:00<?, ?it/s]

Parsed 19 nodes


In [5]:
quantization_conf = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

#We recreate template (or messages) and ensure that they have the correct format, as per,
#https://github.com/run-llama/llama_index/issues/9277#issuecomment-1837545398 for zephyr-7b-beta

def messages_to_prompt(messages):
    prompt = ""
    for message in messages:
        if message.role == 'system':
            prompt += f"<|system|>\n{message.content}</s>\n"
        elif message.role == 'user':
            prompt += f"<|user|>\n{message.content}</s>\n"
        elif message.role == 'assistant':
            prompt += f"<|assistant|>\n{message.content}</s>\n"

    # ensure we start with a system prompt, insert blank if needed
    if not prompt.startswith("<|system|>\n"):
        prompt = "<|system|>\n</s>\n" + prompt

    # add final assistant prompt
    prompt = prompt + "<|assistant|>\n"
    return prompt

In [6]:
!pip install bitsandbytes -U

In [7]:
def huggingface_llm(model_name="HuggingFaceH4/zephyr-7b-beta",
                    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
                    context_window=3900,
                    max_new_tokens=256,
                    quantization_config = quantization_conf
                   ):
    llm = HuggingFaceLLM(
        model_name=model_name,
        tokenizer_name=tokenizer_name,
        query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
        context_window=context_window,
        max_new_tokens=max_new_tokens,
        model_kwargs={"quantization_config": quantization_config},
        # tokenizer_kwargs={},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
        messages_to_prompt=messages_to_prompt,
        device_map="auto",
    )

    return llm

llm = huggingface_llm()

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [8]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [9]:
train_dataset = generate_qa_embedding_pairs(train_nodes, llm)

100%|██████████| 30/30 [05:26<00:00, 10.88s/it]

Final dataset saved.


In [26]:
print("🔹 First 5 Training Samples:")
for i, item in enumerate(iter(train_dataset)):  # Convert to iterable
    print(f"Sample {i+1}: {item}\n")
    if i == 4:  # Stop after printing 5 samples
        break


🔹 First 5 Training Samples:
Sample 1: ('queries', {'85e33030-2342-4828-aec0-b8516fe6342d': 'How do scaling laws differ for upstream loss and downstream task performance in machine translation?', '120ef4d8-b172-4530-b492-c67f355bef25': 'What factors influence the scaling behavior of downstream task performance in machine translation, and how can these insights be used to guide model development and resource allocation? Specifically, how does the choice of pretraining data and its size affect downstream translation quality scores, and what practical insights can be gained from analyzing cases where moderate misalignment causes fluctuation or degradation in scores?', 'c021bf2d-03c9-4cd4-8612-5deb83093910': 'How does the alignment between pretraining data and downstream tasks affect the performance of LLMs in machine translation?', '8d74d48b-a2b5-45ff-880f-91e634ebbe61': 'What scaling laws have been proposed for measuring the downstream performance of LLMs in machine translation, and how d

The error occurs because train_dataset is an instance of EmbeddingQAFinetuneDataset, which does not support direct indexing (train_dataset[:5]).

In [10]:
from llama_index.finetuning import EmbeddingAdapterFinetuneEngine
from llama_index.core.embeddings import resolve_embed_model
import torch

def embedding_model(model="local:avsolatorio/GIST-large-Embedding-v0",
                    model_output_path="model_output_test",
                    bias=True,
                    no_of_epochs=4,
                    verbose=True,
                    optimizer=torch.optim.AdamW,
                    optimizer_params={"lr": 0.01}
                   ):

    base_embed_model = resolve_embed_model(model)
    finetune_engine = EmbeddingAdapterFinetuneEngine(
        train_dataset,
        base_embed_model,
        model_output_path=model_output_path,
        bias=bias,
        epochs=no_of_epochs,
        verbose=verbose,
        optimizer_class=optimizer,
        optimizer_params=optimizer_params
    )

    return finetune_engine

finetune_engine = embedding_model()
finetune_engine.finetune()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

...

> [Epoch 1] Current loss: 2.108328342437744
> [Epoch 1] Current loss: 4.19433069229126
> [Epoch 1] Current loss: 2.8417210578918457
> [Epoch 1] Current loss: 1.9532511234283447
> [Epoch 1] Current loss: 1.5720385313034058
> [Epoch 1] Current loss: 2.070016384124756


Iteration:   0%|          | 0/6 [00:00<?, ?it/s]

> [Epoch 2] Current loss: 1.6648979187011719
> [Epoch 2] Current loss: 2.4918861389160156
> [Epoch 2] Current loss: 2.070411205291748
> [Epoch 2] Current loss: 1.774507761001587
> [Epoch 2] Current loss: 1.1710445880889893
> [Epoch 2] Current loss: 1.8190208673477173


Iteration:   0%|          | 0/6 [00:00<?, ?it/s]

> [Epoch 3] Current loss: 1.5854251384735107
> [Epoch 3] Current loss: 1.8741238117218018
> [Epoch 3] Current loss: 1.8471472263336182
> [Epoch 3] Current loss: 1.6560289859771729
> [Epoch 3] Current loss: 1.0491833686828613
> [Epoch 3] Current loss: 1.729034185409546
> Finished training, saving to model_output_test


In [11]:
finetune_engine.get_finetuned_model()

AdapterEmbeddingModel(model_name='Adapter for avsolatorio/GIST-large-Embedding-v0', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ec4e8cbbd50>, num_workers=None)

In [29]:
val_nodes = load_corpus(VAL_FILES)
val_dataset = generate_qa_embedding_pairs(val_nodes, llm)

Parsing nodes:   0%|          | 0/13 [00:00<?, ?it/s]

Parsed 19 nodes


30it [00:00, ?it/s]

Final dataset saved.


In [32]:
print("🔹 First 5 Validation Samples:")
for i, item in enumerate(iter(val_dataset)):  # Convert to iterable
    print(f"Sample {i+1}: {item}\n")
    if i == 4:  # Stop after printing 5 samples
        break


🔹 First 5 Validation Samples:
Sample 1: ('queries', {'85e33030-2342-4828-aec0-b8516fe6342d': 'How do scaling laws differ for upstream loss and downstream task performance in machine translation?', '120ef4d8-b172-4530-b492-c67f355bef25': 'What factors influence the scaling behavior of downstream task performance in machine translation, and how can these insights be used to guide model development and resource allocation? Specifically, how does the choice of pretraining data and its size affect downstream translation quality scores, and what practical insights can be gained from analyzing cases where moderate misalignment causes fluctuation or degradation in scores?', 'c021bf2d-03c9-4cd4-8612-5deb83093910': 'How does the alignment between pretraining data and downstream tasks affect the performance of LLMs in machine translation?', '8d74d48b-a2b5-45ff-880f-91e634ebbe61': 'What scaling laws have been proposed for measuring the downstream performance of LLMs in machine translation, and how

In [13]:
print(type(val_dataset))
print(dir(val_dataset))  # Lists all available methods & attributes


<class 'llama_index.finetuning.embeddings.common.EmbeddingQAFinetuneDataset'>
['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer

In [14]:
for i, item in enumerate(val_dataset):
    print(item)
    if i == 4:  # Stop after 5 entries
        break


('queries', {'85e33030-2342-4828-aec0-b8516fe6342d': 'How do scaling laws differ for upstream loss and downstream task performance in machine translation?', '120ef4d8-b172-4530-b492-c67f355bef25': 'What factors influence the scaling behavior of downstream task performance in machine translation, and how can these insights be used to guide model development and resource allocation? Specifically, how does the choice of pretraining data and its size affect downstream translation quality scores, and what practical insights can be gained from analyzing cases where moderate misalignment causes fluctuation or degradation in scores?', 'c021bf2d-03c9-4cd4-8612-5deb83093910': 'How does the alignment between pretraining data and downstream tasks affect the performance of LLMs in machine translation?', '8d74d48b-a2b5-45ff-880f-91e634ebbe61': 'What scaling laws have been proposed for measuring the downstream performance of LLMs in machine translation, and how do they differ based on the alignment b

In [15]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

In [16]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [17]:
#pre-train GIST-large-Embedding-v0
gist_embed_model = "local:avsolatorio/GIST-large-Embedding-v0"
gist_val_results = evaluate(train_dataset, gist_embed_model)

# Create DataFrame using the correctly named variable gist_val_results
df_embed_models = pd.DataFrame(gist_val_results)
hit_rate_bge = df_embed_models["is_hit"].mean()
print(f"Hit Rate: {hit_rate_bge:.4f}")  # Print hit rate with 4 decimal places


Generating embeddings:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Hit Rate: 0.8000


In [21]:
# Fine-tuned model DataFrame (SEPARATE)
#fine-tuned GIST-large-Embedding-v0
ft_gist_embed_model = finetune_engine.get_finetuned_model()
ft_gist_val_results = evaluate(train_dataset, ft_gist_embed_model)
df_ft_embed_models = pd.DataFrame(ft_gist_val_results)
hit_rate_ft = df_ft_embed_models["is_hit"].mean()
print(f"Fine-tuned Hit Rate: {hit_rate_ft:.4f}")


Generating embeddings:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Fine-tuned Hit Rate: 0.5500


In [22]:
# Pre-trained model DataFrame
df_embed_models = pd.DataFrame(gist_val_results)
hit_rate_bge = df_embed_models["is_hit"].mean()
print(f"Pre-trained Hit Rate: {hit_rate_bge:.4f}")

# Fine-tuned model DataFrame (SEPARATE)
df_ft_embed_models = pd.DataFrame(ft_gist_val_results)
hit_rate_ft = df_ft_embed_models["is_hit"].mean()
print(f"Fine-tuned Hit Rate: {hit_rate_ft:.4f}")

# Compare both DataFrames
print(df_embed_models.head())  # Check first few rows
print(df_ft_embed_models.head())  # Check first few rows


Pre-trained Hit Rate: 0.8000
Fine-tuned Hit Rate: 0.5500
   is_hit                                          retrieved  \
0    True  [904e0ddb-d5e7-47e3-9154-c31713df174a, 2742a2e...   
1    True  [904e0ddb-d5e7-47e3-9154-c31713df174a, 34d04ca...   
2    True  [34d04ca3-1bea-44c8-beba-492dda382655, e20c819...   
3    True  [904e0ddb-d5e7-47e3-9154-c31713df174a, b870900...   
4   False  [9bf4dc1a-441b-443b-bcc4-100226077eca, 904e0dd...   

                               expected                                 query  
0  904e0ddb-d5e7-47e3-9154-c31713df174a  85e33030-2342-4828-aec0-b8516fe6342d  
1  904e0ddb-d5e7-47e3-9154-c31713df174a  120ef4d8-b172-4530-b492-c67f355bef25  
2  34d04ca3-1bea-44c8-beba-492dda382655  c021bf2d-03c9-4cd4-8612-5deb83093910  
3  34d04ca3-1bea-44c8-beba-492dda382655  8d74d48b-a2b5-45ff-880f-91e634ebbe61  
4  b870900a-454b-4906-b543-43dfe4155d3e  0872fad5-897d-474e-bb57-38ab2a8eb6b2  
   is_hit                                          retrieved  \
0   False  [44

In [20]:
#pre-train GIST-large-Embedding-v0
gist_embed_model = "local:avsolatorio/GIST-large-Embedding-v0"
gist_val_results = evaluate(train_dataset, gist_embed_model)

# Create DataFrame using the correctly named variable gist_val_results
df_embed_models = pd.DataFrame(gist_val_results)
hit_rate_bge = df_embed_models["is_hit"].mean()
print(f"Hit Rate: {hit_rate_bge:.4f}")  # Print hit rate with 4 decimal places

# Evaluate the fine-tuned model using the 'evaluate' function
# and assign the results to 'ft_gist_val_results'
ft_gist_embed_model = finetune_engine.get_finetuned_model() # Get the fine-tuned model
ft_gist_val_results = evaluate(val_dataset, ft_gist_embed_model) # Evaluate

# Fine-tuned model DataFrame (SEPARATE)
df_ft_embed_models = pd.DataFrame(ft_gist_val_results)
hit_rate_ft = df_ft_embed_models["is_hit"].mean()
print(f"Fine-tuned Hit Rate: {hit_rate_ft:.4f}")

Generating embeddings:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Hit Rate: 0.8000


Generating embeddings:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Fine-tuned Hit Rate: 0.5500


In [33]:
print("\n🔹 Fine-Tuned Model Evaluation Results (First 5):")
print(df_ft_embed_models.head())
#After fine-tuning, you can inspect how the retrieved results compare with expected results.


🔹 Fine-Tuned Model Evaluation Results (First 5):
   is_hit                                          retrieved  \
0   False  [44529355-c0d5-4d59-aedf-424382a76696, 36ee642...   
1   False  [44529355-c0d5-4d59-aedf-424382a76696, 69a426f...   
2   False  [44529355-c0d5-4d59-aedf-424382a76696, ca07850...   
3   False  [44529355-c0d5-4d59-aedf-424382a76696, 36ee642...   
4    True  [1558cbd6-f258-40ec-b388-fd05002b6a87, 4452935...   

                               expected                                 query  
0  904e0ddb-d5e7-47e3-9154-c31713df174a  85e33030-2342-4828-aec0-b8516fe6342d  
1  904e0ddb-d5e7-47e3-9154-c31713df174a  120ef4d8-b172-4530-b492-c67f355bef25  
2  34d04ca3-1bea-44c8-beba-492dda382655  c021bf2d-03c9-4cd4-8612-5deb83093910  
3  34d04ca3-1bea-44c8-beba-492dda382655  8d74d48b-a2b5-45ff-880f-91e634ebbe61  
4  b870900a-454b-4906-b543-43dfe4155d3e  0872fad5-897d-474e-bb57-38ab2a8eb6b2  
